In [43]:
from KnowledgeGraph import KnowledgeGraph
from schema import *
from helper import *

### 1. Retrieve knowledge from graph

In [44]:
# Create knowledge graph object
kg = KnowledgeGraph("intellikgraph")

In [45]:
# Retrieve list of vessels
vessels = kg.extract_vessels()
vessels

['Vessel_E', 'Vessel_D', 'Vessel_C', 'Vessel_B', 'Vessel_A']

In this example, we will identify potential behavior events for Vessel A

In [46]:
# Find nearby vessels and related events
neighbors = kg.find_nearby_vessels("Vessel_A")
events = kg.find_related_events("Vessel_A")
print(events)
print(neighbors)

([], [], [], ['wx_WCPFC_01'])
['Vessel_C']


Vessel A is close to Vessel C and related to event wx_WCPFC_01

### 2. Compile knowledge into JSON schema

In [53]:
# Create a cluster of a vessel and nearby vessels
cluster = construct_cluster("Vessel_A", neighbors, events, kg)
cluster_string = cluster.model_dump_json()

# Reformat timestamps
import re
cluster_string = re.sub(r"\d{2}\+\d{2}:\d{2}", "", cluster_string)
cluster_string

'{"vessel":{"id":"Vessel_A","name":"SEAFARER I","type":"SmallTrawler","flag":"PH","observed_points":[{"timestamp":"2025-10-04 23:30:","lat":14.2,"lon":121.2,"speed_knots":9.2,"course_degrees":195.0,"dist_from_port_km":40.0,"dist_from_shore_km":15.0},{"timestamp":"2025-10-05 06:00:","lat":14.45,"lon":121.8,"speed_knots":8.7,"course_degrees":200.0,"dist_from_port_km":90.0,"dist_from_shore_km":40.0},{"timestamp":"2025-10-05 13:45:","lat":11.9,"lon":159.8,"speed_knots":9.1,"course_degrees":80.0,"dist_from_port_km":700.0,"dist_from_shore_km":300.0},{"timestamp":"2025-10-05 14:10:","lat":12.1,"lon":160.1,"speed_knots":7.5,"course_degrees":20.0,"dist_from_port_km":710.0,"dist_from_shore_km":310.0},{"timestamp":"2025-10-05 14:20:","lat":12.25,"lon":160.15,"speed_knots":2.0,"course_degrees":350.0,"dist_from_port_km":712.0,"dist_from_shore_km":312.0},{"timestamp":"2025-10-05 14:30:","lat":12.35,"lon":160.2,"speed_knots":2.1,"course_degrees":355.0,"dist_from_port_km":714.0,"dist_from_shore_km":31

### 3. Instruct LLM to generate behavior events

In [6]:
from openai import OpenAI
client = OpenAI()

In [7]:
SYSTEM_PROMPT = """
    You are a data scientist at a maritime research company. 
    You are given data in JSON format about a particular vessel and its neighboring vessels.
    Your task is to identify potential encounter, loitering, and course deviation behaviors for this particular vessel.
    Do not make any unreasonable assumptions nor use evidence beyond the given data.
    If fields in the output (like observed_points and predicted_points) can be inferred from the input, use input values to populate these fields.
    In your explanations, make sure any input data referenced exactly matches that of the input.
"""

In [8]:
res = client.beta.chat.completions.parse(
    model = "gpt-5",
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Data: {cluster_string}"}
    ],
    response_format = Response
)

In [ ]:
loit_1 = res.choices[0].message.parsed.loitering_events[0]
print(f"Loitering event: {loit_1.model_dump_json()}")
print(f"Explanation: {loit_1.explanation}")

{"id":"loiter_001","type":"Loitering","start_time":"2025-10-05 14:20:","end_time":"2025-10-05 14:30:","location":"~12.25N, 160.15E to 12.35N, 160.20E (open ocean, ~312–314 km from shore)","vessel":{"id":"Vessel_A","name":"SEAFARER I","type":"SmallTrawler","flag":"PH","observed_points":[{"timestamp":"2025-10-04 23:30:","lat":14.2,"lon":121.2,"speed_knots":9.2,"course_degrees":195.0,"dist_from_port_km":40.0,"dist_from_shore_km":15.0},{"timestamp":"2025-10-05 06:00:","lat":14.45,"lon":121.8,"speed_knots":8.7,"course_degrees":200.0,"dist_from_port_km":90.0,"dist_from_shore_km":40.0},{"timestamp":"2025-10-05 13:45:","lat":11.9,"lon":159.8,"speed_knots":9.1,"course_degrees":80.0,"dist_from_port_km":700.0,"dist_from_shore_km":300.0},{"timestamp":"2025-10-05 14:10:","lat":12.1,"lon":160.1,"speed_knots":7.5,"course_degrees":20.0,"dist_from_port_km":710.0,"dist_from_shore_km":310.0},{"timestamp":"2025-10-05 14:20:","lat":12.25,"lon":160.15,"speed_knots":2.0,"course_degrees":350.0,"dist_from_port

### 4. Evaluate precision of explanation

In [ ]:
# Generate atomic facts for explanation and cluster
explanation_facts = extract_facts(loit_1.explanation, client)
cluster_facts = extract_facts(cluster_string, client)
print(f"Explanation facts: {explanation_facts}")
print(f"Cluster facts: {cluster_facts}")

In [ ]:
check = fact_check(loit_1.explanation, cluster_string, client)
print(f"Precision: {precision(check[0], check[1])}")

1.0

### 5. Filter for important events & explain why they are important

See 'filtering_guidelines' in data folder